In [2]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import re
import requests
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt


/Users/jeffreyfang/is310-fall-2024-group-3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
spotify = pd.read_csv("Billboard_Hot100_Songs_Spotify_1946-2022.csv")

In [5]:


dff = pd.read_csv("songslyrics.csv")

def remove_short_words(text):
    return re.sub(r'\b\w{1,3}\b', '', text)

# Apply the function to the 'Lyrics' column
dff['Lyrics'] = dff['Lyrics'].apply(lambda x: remove_short_words(x) if isinstance(x, str) else x)
lyrics = dff['Lyrics'].dropna().tolist()
print(len(lyrics))
topic_model = BERTopic(min_topic_size=5, nr_topics=30)

topics, probabilities = topic_model.fit_transform(lyrics)
vectorizer_model = CountVectorizer(stop_words="english")
topic_model.update_topics(lyrics, vectorizer_model=vectorizer_model)

topic_info = topic_model.get_topic_info()

print(topic_info)

241


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


    Topic  Count                             Name  \
0      -1     71           -1_like_know_girl_love   
1       0      9    0_poker_read_battlefield_face   
2       1      5        1_says_word_diamonds_hear   
3       2      5  2_alabama_singing_stress_summer   
4       3      8          3_said_gonna_know_rehab   
5       4     28           4_walk_kiss_want_dance   
6       5      6         5_cold_whatcha_stop_just   
7       6     10        6_locked_love_lose_wanted   
8       7     59           7_time_love_cause_fine   
9       8      7    8_womanizer_stuntin_diva_just   
10      9     20          9_like_money_rock_watch   
11     10     13         10_like_shawty_good_know   

                                       Representation  \
0   [like, know, girl, love, yeah, away, shake, wa...   
1   [poker, read, battlefield, face, armor, girlfr...   
2   [says, word, diamonds, hear, jerusalem, ruled,...   
3   [alabama, singing, stress, summer, sweet, girl...   
4   [said, gonna, know, r

In [6]:


# Function to get lyrics using the lyrics.ovh API
def get_lyrics(song_title, artist_name):
    url = f"https://api.lyrics.ovh/v1/{artist_name}/{song_title}"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            return data.get('lyrics', 'Lyrics not found')
        except ValueError:
            return 'Lyrics not found'
    else:
        return 'Lyrics not found'

# Function to clean the 'Artist Names' column by removing brackets, quotes, and characters after the first comma
def extract_artist_name(artist_string):
    artist_string = artist_string.replace("[", "").replace("]", "").replace("'", "").replace('"', "").strip()
    return artist_string.split(",")[0].strip()

# Function to filter DataFrame by years and retrieve lyrics
def filter_and_retrieve_lyrics(df, start_year, end_year):
    # Filter the DataFrame based on the 'Hot100 Ranking Year' column
    filtered_df = df[(df['Hot100 Ranking Year'] >= start_year) & (df['Hot100 Ranking Year'] <= end_year)].copy()
    
    # Initialize the 'Lyrics' column
    filtered_df['Lyrics'] = ''
    
    # Loop through the filtered DataFrame and retrieve lyrics
    for index, row in filtered_df.iterrows():
        song_title = row['Song']
        row["Artist Names"] = extract_artist_name(row["Artist Names"])
        artist_name = row['Artist Names']
        print(row["Artist Names"], row['Song'])
        if artist_name:
            lyrics = get_lyrics(song_title, artist_name)
            filtered_df.at[index, 'Lyrics'] = lyrics
        else:
            filtered_df.at[index, 'Lyrics'] = 'Artist not found'
    
    return filtered_df



In [54]:
# Example usage
# Assuming reccesion_songs is your DataFrame
start_year = 2000
end_year = 2004
result_df = filter_and_retrieve_lyrics(spotify, start_year, end_year)

# Display the resulting DataFrame
result_df

Faith Hill Breathe
Santana Smooth (feat. Rob Thomas)
Santana Maria Maria (feat. The Product G&B)
Joe I Wanna Know
Vertical Horizon Everything You Want
Destinys Child Say My Name
Savage Garden I Knew I Loved You
Lonestar Amazed
Matchbox Twenty Bent
Toni Braxton He Wasn't Man Enough
Creed Higher
Aaliyah Try Again
Destinys Child Jumpin', Jumpin'
Sisqo Thong Song
3 Doors Down Kryptonite
The Pinkertons I Met You There
Madonna Music
Janet Jackson Doesn't Really Matter
Christina Aguilera What a Girl Wants
Brian McKnight Back At One
*NSYNC Bye Bye Bye
Marc Anthony You Sang To Me
Marc Anthony I Need to Know
Montell Jordan Get It On Tonite
Sisqo Incomplete
Macy Gray I Try
*NSYNC It's Gonna Be Me
Céline Dion That's the Way It Is
Nelly Country Grammar (Hot Shit)
Blaque Bring It All to Me (feat. *NSYNC)
Backstreet Boys Show Me the Meaning of Being Lonely
Missy Elliott Hot Boyz
BBMAK Back Here
Sonique It Feels So Good
Nine Days Absolutely (Story of a Girl) - Radio Mix
Creed With Arms Wide Open
Enriq

,Song,Album,Album Release Date,Artist Names,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank,Song Length(ms),Spotify Link,Song Image,...,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Key,Mode,Time Signature,Lyrics
4585,Breathe,Breathe,1999-11-09,['Faith Hill'],"['country dawn', 'contemporary country', 'coun...",2000,1,250546,https://open.spotify.com/track/3y4LxiYMgDl4Ret...,https://i.scdn.co/image/ab67616d00001e02f4adbb...,...,0.000000,0.2510,-9.007,0.0290,136.859,0.278,7,1,4,I can feel magic floating in the air\r\nBeing ...
4586,Smooth (feat. Rob Thomas),Supernatural (Remastered),1999-06-15,"['Santana', 'Rob Thomas']","['mexican classic rock', 'neo mellow', 'post-g...",2000,2,294986,https://open.spotify.com/track/0n2SEXB2qoRQg17...,https://i.scdn.co/image/ab67616d00001e0247eb3e...,...,0.000005,0.2950,-3.908,0.0338,115.996,0.961,9,1,4,"Man, it's a hot one\r\nLike seven inches from ..."
4587,Maria Maria (feat. The Product G&B),Supernatural (Remastered),1999-06-15,"['Santana', 'The Product G&B']","['classic rock', 'blues rock', 'mexican classi...",2000,3,261973,https://open.spotify.com/track/3XKIUb7HzIF1Vu9...,https://i.scdn.co/image/ab67616d00001e0247eb3e...,...,0.002010,0.0348,-5.931,0.1260,97.911,0.680,2,1,4,Ho Maria Maria\r\nShe reminds me of a west sid...
4588,I Wanna Know,My Name Is Joe,2000-04-18,['Joe'],"['contemporary r&b', 'urban contemporary', 'r&b']",2000,4,296693,https://open.spotify.com/track/1m2xMsxbtxv21Br...,https://i.scdn.co/image/ab67616d00001e02a922b8...,...,0.000011,0.4310,-5.959,0.0368,136.086,0.599,8,0,4,"Yeah, oh yeah\r\nAlright, oh, oh, oh\r\nIt's a..."
4589,Everything You Want,Everything You Want,1999-06-14,['Vertical Horizon'],"['neo mellow', 'pop rock', 'post-grunge']",2000,5,257066,https://open.spotify.com/track/4cKGldbhGJniI8B...,https://i.scdn.co/image/ab67616d00001e02dde37a...,...,0.000000,0.0642,-6.389,0.0306,93.896,0.736,3,0,4,Somewhere there's speaking\r\nIt's already com...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,Over And Over,Suit,2004-09-14,"['Nelly', 'Tim McGraw']","['gangster rap', 'dance pop', 'country road', ...",2004,96,253933,https://open.spotify.com/track/2b8QQqf8Vwb7bfz...,https://i.scdn.co/image/ab67616d00001e026ff5d0...,...,0.000001,0.1580,-10.020,0.0803,169.799,0.464,7,1,4,Cause its all in my head\r\nI think about it o...
5081,Redneck Woman,Here For The Party,2004-05-11,['Gretchen Wilson'],"['country dawn', 'contemporary country', 'coun...",2004,97,221333,https://open.spotify.com/track/26bL4gSULWDgdIM...,https://i.scdn.co/image/ab67616d00001e02ddf152...,...,0.000000,0.3060,-5.146,0.1770,185.069,0.753,6,1,4,"Well, I ain't never been the Barbie doll type\..."
5082,Just Lose It,Encore (Deluxe Version),2004-11-12,['Eminem'],"['hip hop', 'rap', 'detroit hip hop']",2004,98,248680,https://open.spotify.com/track/0qcjuYtMWhBjXg0...,https://i.scdn.co/image/ab67616d00001e02726d48...,...,0.000040,0.2810,-3.560,0.0467,121.003,0.962,8,1,4,Down down down\r\nDown down down\r\nDown down ...
5083,Letters from Home,Letters From Home,2004-04-20,['John Michael Montgomery'],"['contemporary country', 'country road', 'coun...",2004,99,267533,https://open.spotify.com/track/15cGdiAHmezm0P0...,https://i.scdn.co/image/ab67616d00001e02d0b384...,...,0.000006,0.1550,-7.243,0.0258,128.029,0.388,0,1,4,"My Dear Son, it's almost June\r\nI hope this l..."


In [55]:
result_df = result_df[result_df['Lyrics'] != 'Lyrics not found']

In [53]:
result_df.to_clipboard(index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:

dff = result_df

def remove_short_words(text):
    return re.sub(r'\b\w{2,3}\b', '', text)

# Apply the function to the 'Lyrics' column
dff['Lyrics'] = dff['Lyrics'].apply(lambda x: remove_short_words(x) if isinstance(x, str) else x)
lyrics = dff['Lyrics'].dropna().tolist()
print(len(lyrics))
topic_model = BERTopic(min_topic_size=5, nr_topics=15)
topics, probabilities = topic_model.fit_transform(lyrics)
stop_words = [
    'i', "im", "i'm" , "a", 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
    'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
    'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"

]
profanities = []
with open('profanity.txt', 'r') as file:
    prof_string = file.read().replace('\n', '')
    prof_tokens = prof_string.split(",")
    for token in prof_tokens:
        profanities.append(token)
stop_words.extend(profanities)
print(stop_words)

vectorizer_model = CountVectorizer(stop_words=stop_words)
topic_model.update_topics(lyrics, vectorizer_model=vectorizer_model)

topic_info = topic_model.get_topic_info()

print(topic_info)


401
['i', 'im', "i'm", 'a', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just'

<bound method NDFrame.to_clipboard of                                      Song                      Album  \
4585                              Breathe                    Breathe   
4586            Smooth (feat. Rob Thomas)  Supernatural (Remastered)   
4587  Maria Maria (feat. The Product G&B)  Supernatural (Remastered)   
4588                         I Wanna Know             My Name Is Joe   
4589                  Everything You Want        Everything You Want   
...                                   ...                        ...   
5080                        Over And Over                       Suit   
5081                        Redneck Woman         Here For The Party   
5082                         Just Lose It    Encore (Deluxe Version)   
5083                    Letters from Home          Letters From Home   
5084                            I Go Back     When The Sun Goes Down   

     Album Release Date                    Artist Names  \
4585         1999-11-09               

In [57]:
print(dff["Lyrics"])

4585    I  feel magic floating   \r\nBeing with  gets ...
4586    , 's a  \r\nLike seven inches from  midday \r\...
4587     Maria Maria\r\n reminds   a west side story\r...
4588    Yeah,  yeah\r\nAlright, , , \r\n's amazing   k...
4589    Somewhere there's speaking\r\n's already comin...
                              ...                        
5080    Cause     head\r\nI think about  over  over ag...
5081    Well, I 't never been  Barbie doll type\r\n, I...
5082    Down down down\r\nDown down down\r\nDown down ...
5083     Dear , 's almost June\r\nI hope this letter c...
5084    "Jack  Diane" painted a picture   life   dream...
Name: Lyrics, Length: 401, dtype: object


In [87]:
topic_model.visualize_barchart(width=280, height=330, top_n_topics=15, n_words=20)

In [ ]:
topic_model.visualize_topics()
topic_model.visualize_documents(lyrics,
# These were calculated beforehand with UMAP
topics=list(range (30)), custom_labels=True, height=600)

In [89]:
topic_model.visualize_documents(lyrics,
# These were calculated beforehand with UMAP
topics=list(range (30)), custom_labels=True, height=600)

In [94]:
# Example usage
# Assuming reccesion_songs is your DataFrame
start_year = 2005
end_year = 2006
result_df = filter_and_retrieve_lyrics(spotify, start_year, end_year)

# Display the resulting DataFrame
result_df

Mariah Carey We Belong Together
Gwen Stefani Hollaback Girl
Mario Let Me Love You
Kelly Clarkson Since U Been Gone
Ciara 1, 2 Step (feat. Missy Elliott) - Don Candiani Reggaeton Mix
Kanye West Gold Digger
Green Day Boulevard of Broken Dreams
50 Cent Candy Shop
The Pussycat Dolls Don't Cha
Kelly Clarkson Behind These Hazel Eyes
50 Cent Disco Inferno
Lifehouse You And Me
AVID All Stars Don't Phunk With My Heart (In The Style Of Black Eyed Peas)
Missy Elliott Lose Control (feat. Ciara & Fat Man Scoop)
Mariah Carey Shake It Off
The Killers Mr. Brightside
50 Cent Just A Lil Bit
Rihanna Pon de Replay
The Karaoke Crew How We Do (As Made Famous By The Game Featuring. 50 Cent)
Weezer Beverly Hills
Ciara Oh (feat. Ludacris)
Rob Thomas Lonely No More
Snoop Dogg Drop It Like It's Hot
The Game Hate It Or Love It
Lil Jon & The East Side Boyz Lovers And Friends
Destinys Child Soldier (feat. T.I. & Lil' Wayne)
Kelly Clarkson Breakaway
Will Smith Switch
Bow Wow Let Me Hold You (feat. Omarion)
Bow Wow L

,Song,Album,Album Release Date,Artist Names,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank,Song Length(ms),Spotify Link,Song Image,...,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Key,Mode,Time Signature,Lyrics
5085,We Belong Together,The Emancipation of Mimi,2005-04-12,['Mariah Carey'],"['urban contemporary', 'dance pop', 'pop']",2005,1,201400,https://open.spotify.com/track/3LmvfNUQtglbTry...,https://i.scdn.co/image/ab67616d00001e02768bc8...,...,0.000000,0.0900,-7.890,0.0601,139.986,0.762,0,1,4,I didn't mean it\nWhen I said I didn't love yo...
5086,Hollaback Girl,Love Angel Music Baby (Deluxe Version),2004-11-23,['Gwen Stefani'],"['dance pop', 'pop']",2005,2,199853,https://open.spotify.com/track/0LzrhCZFXW94Y8n...,https://i.scdn.co/image/ab67616d00001e027c7136...,...,0.000006,0.0234,-2.221,0.0929,110.007,0.904,10,0,4,"Uh huh, this is my shit\r\nAll the girls stomp..."
5087,Let Me Love You,Let Me Love You,2004-09-21,['Mario'],"['southern hip hop', 'dance pop', 'urban conte...",2005,3,256733,https://open.spotify.com/track/3ibKnFDaa3GhpPG...,https://i.scdn.co/image/ab67616d00001e02c42212...,...,0.000000,0.1180,-8.970,0.0922,94.514,0.556,7,0,4,"Mmmm ..... Mmmmm.... Yeah....Mmmmm....Yeah, Ye..."
5088,Since U Been Gone,Breakaway,2004-01-17,['Kelly Clarkson'],"['talent show', 'dance pop', 'pop']",2005,4,188960,https://open.spotify.com/track/3xrn9i8zhNZsTtc...,https://i.scdn.co/image/ab67616d00001e0203dadd...,...,0.030300,0.1140,-5.406,0.0334,131.000,0.404,0,1,4,Here's the thing we started out friends\r\nIt ...
5089,"1, 2 Step (feat. Missy Elliott) - Don Candiani...",DVD Bonus Audio (from Goodies. The Videos and ...,2005-07-12,"['Ciara', 'Missy Elliott']","['hip pop', 'dance pop', 'urban contemporary',...",2005,5,231773,https://open.spotify.com/track/3x9LFpV6tzq0qWa...,https://i.scdn.co/image/ab67616d00001e02abc845...,...,0.002930,0.3360,-2.530,0.0414,98.738,0.776,7,0,4,Lyrics not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,Stupid Girls (Karaoke Version) - Originally Pe...,Remember Me - 2006,2012-01-01,['Sing Karaoke Sing'],['karaoke'],2006,96,195030,https://open.spotify.com/track/6h5heo2YVPH60c0...,https://i.scdn.co/image/ab67616d00001e0258c6d4...,...,0.000008,0.1080,-9.500,0.0484,100.053,0.757,4,0,4,Lyrics not found
5280,Feel Good Inc.,Demon Days,2005-05-23,['Gorillaz'],"['alternative hip hop', 'modern rock', 'rock']",2006,97,222640,https://open.spotify.com/track/0d28khcov6AiegS...,https://i.scdn.co/image/ab67616d00001e0219d85a...,...,0.002330,0.6130,-6.679,0.1770,138.559,0.772,6,1,4,Hahahahahahahahahahaha City's breaking down on...
5281,I'm Sprung,Rappa Ternt Sanga (Expanded Edition),2005-12-06,['T-Pain'],"['gangster rap', 'southern hip hop', 'dance po...",2006,98,231040,https://open.spotify.com/track/0DLOyyQvwPTSDKu...,https://i.scdn.co/image/ab67616d00001e02dbaa4f...,...,0.000000,0.1030,-11.596,0.0999,100.000,0.195,0,0,4,[Chorus]\r\nI'm Sprung...(I'm sprung)\r\nDawg ...
5282,Do I Make You Proud,Do I Make You Proud / Takin' It To The Streets,2006-06-13,['Taylor Hicks'],['idol'],2006,99,248693,https://open.spotify.com/track/3lxFENvLW8nxXAb...,https://i.scdn.co/image/ab67616d00001e02000a45...,...,0.000000,0.1020,-8.207,0.0316,144.055,0.308,1,1,4,[Verse One:]\r\nI've never been\r\nThe one to ...


In [95]:
result_df1 = result_df[result_df['Lyrics'] != 'Lyrics not found']
result_df.to_clipboard(index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [96]:

dff = result_df1

def remove_short_words(text):
    return re.sub(r'\b\w{2,3}\b', '', text)

# Apply the function to the 'Lyrics' column
dff['Lyrics'] = dff['Lyrics'].apply(lambda x: remove_short_words(x) if isinstance(x, str) else x)
lyrics = dff['Lyrics'].dropna().tolist()
print(len(lyrics))
topic_model = BERTopic(min_topic_size=5, nr_topics=15)
topics, probabilities = topic_model.fit_transform(lyrics)
stop_words = [
    'i', "im", "i'm" , "a", 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
    'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
    'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"

]
profanities = []
with open('profanity.txt', 'r') as file:
    prof_string = file.read().replace('\n', '')
    prof_tokens = prof_string.split(",")
    for token in prof_tokens:
        profanities.append(token)
stop_words.extend(profanities)
print(stop_words)

vectorizer_model = CountVectorizer(stop_words=stop_words)
topic_model.update_topics(lyrics, vectorizer_model=vectorizer_model)

topic_info = topic_model.get_topic_info()

print(topic_info)


148
['i', 'im', "i'm", 'a', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just'

In [29]:
dff.to_clipboard(index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [98]:
topic_model.visualize_barchart(width=280, height=330, top_n_topics=8, n_words=20)


In [99]:
topic_model.visualize_topics()

In [ ]:

topic_model.visualize_documents(lyrics,
# These were calculated beforehand with UMAP
topics=list(range (30)), custom_labels=True, height=600)


In [101]:
# Example usage
# Assuming reccesion_songs is your DataFrame
spotify = pd.read_csv('reccesion_songs.csv')
start_year = 2007
end_year = 2009
result_df = filter_and_retrieve_lyrics(spotify, start_year, end_year)

# Display the resulting DataFrame
result_df

Beyoncé Irreplaceable
Rihanna Umbrella
Gwen Stefani The Sweet Escape
Fergie Big Girls Don't Cry (Personal)
T-Pain Buy U a Drank (Shawty Snappin') (feat. Yung Joc)
Carrie Underwood Before He Cheats
Plain White Ts Hey There Delilah
Akon I Wanna Love You
Nelly Furtado Say It Right
Fergie Glamorous
Akon Don’t Matter
Avril Lavigne Girlfriend
Maroon 5 Makes Me Wonder
Shop Boyz Party Like A Rock Star
Akon Smack That
MiMS This Is Why I'm Hot
Daughtry It's Not Over
Timbaland The Way I Are
Fergie Glamorous
Soulja Boy Crank That (Soulja Boy)
Timbaland Give It To Me
Justin Timberlake What Goes Around.../...Comes Around (Interlude)
Gym Class Heroes Cupid's Chokehold / Breakfast in America
The Fray How to Save a Life
Daughtry Home
Justin Timberlake My Love (feat. T.I.)
Kanye West Stronger
Jim Jones We Fly High
Pink U & Ur Hand
Unk Walk It Out
Sean Kingston Beautiful Girls
Fall Out Boy This Ain't A Scene, It's An Arms Race
T-Pain Bartender (feat. Akon)
Huey Pop, Lock & Drop It - Video Edit
Lil Peep R

,Song,Album,Album Release Date,Artist Names,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank,Song Length(ms),Spotify Link,Song Image,...,Liveness,Loudness,Speechiness,Tempo,Valence,Key,Mode,Time Signature,Lyrics,Release Year
0,Irreplaceable,B'Day Deluxe Edition,2007-05-29,['Beyoncé'],"['pop', 'r&b']",2007,1,227853,https://open.spotify.com/track/6RX5iL93VZ5fKmy...,https://i.scdn.co/image/ab67616d00001e02026e88...,...,0.1670,-4.637,0.3820,175.868,0.509,7,0,4,To the left\r\nTo the left\r\nTo the left\r\nT...,2007.0
1,Umbrella,Speedy Kiwi,2022-08-03,['Rihanna'],['sped up'],2007,2,82128,https://open.spotify.com/track/1nDB6w3ZdYEzzIi...,https://i.scdn.co/image/ab67616d00001e0242a53c...,...,0.0904,-8.555,0.0338,96.513,0.113,9,1,4,"No clouds in my stones\r\nLet it rain, I hydro...",2022.0
2,The Sweet Escape,The Sweet Escape,2006-01-01,"['Gwen Stefani', 'Akon']","['dance pop', 'pop']",2007,3,246466,https://open.spotify.com/track/66ZcOcouenzZEnz...,https://i.scdn.co/image/ab67616d00001e028dd53d...,...,0.1780,-3.502,0.0343,119.961,0.730,1,1,4,"If I could escape \nI would but, first of all,...",2006.0
3,Big Girls Don't Cry (Personal),The Dutchess,2006-09-13,['Fergie'],"['dance pop', 'pop']",2007,4,268120,https://open.spotify.com/track/4AniPkv5vgdE1n6...,https://i.scdn.co/image/ab67616d00001e020283b8...,...,0.0925,-4.271,0.0337,113.083,0.249,7,1,4,"La, da, da, da\r\nThe smell of your skin linge...",2006.0
4,Buy U a Drank (Shawty Snappin') (feat. Yung Joc),Epiphany (Expanded Edition),2007-06-05,"['T-Pain', 'Yung Joc']","['gangster rap', 'southern hip hop', 'dance po...",2007,5,227960,https://open.spotify.com/track/1tkMs9wDPknQ3ZH...,https://i.scdn.co/image/ab67616d00001e02baf2f3...,...,0.0677,-8.141,0.1190,80.028,0.605,1,1,4,Lyrics not found,2007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Goodbye,Exposed (Deluxe Edition),2009-01-01,['Kristinia DeBarge'],['deep dance pop'],2009,96,208133,https://open.spotify.com/track/5myYDbAurm1CW03...,https://i.scdn.co/image/ab67616d00001e0236b57d...,...,0.1420,-3.211,0.0471,119.987,0.651,1,0,4,Am I supposed to put my life on hold because y...,2009.0
296,Say Hey (I Love You) (feat. Cherine Tanya Ande...,All Rebel Rockers,2008-09-09,"['Michael Franti & Spearhead', 'Cherine Tanya ...",['reggae fusion'],2009,97,235760,https://open.spotify.com/track/383B8ksM58jbV3H...,https://i.scdn.co/image/ab67616d00001e024eef40...,...,0.1830,-4.374,0.0852,92.998,0.957,5,0,4,Lyrics not found,2008.0
297,Pop Champagne (feat. Juelz Santana) - Explicit...,Pray IV Reign,2009-03-19,"['Jim Jones', 'Ron Browz', 'Juelz Santana']","['nyc rap', 'harlem hip hop', 'trap', 'gangste...",2009,98,215973,https://open.spotify.com/track/5lrOsw7J8R7shum...,https://i.scdn.co/image/ab67616d00001e02a47a66...,...,0.0990,-6.730,0.2940,136.878,0.463,0,1,3,Lyrics not found,2009.0
298,Pretty Wings - uncut,BLACKsummers'night (2009),2009-07-07,['Maxwell'],"['urban contemporary', 'neo soul', 'r&b', 'qui...",2009,99,310520,https://open.spotify.com/track/22NLm3IIR9NLG0c...,https://i.scdn.co/image/ab67616d00001e023dc772...,...,0.0886,-10.496,0.0668,115.561,0.200,9,1,4,Lyrics not found,2009.0


In [103]:
result_df2 = result_df[result_df['Lyrics'] != 'Lyrics not found']
result_df.to_clipboard(index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [106]:

dff = result_df2

def remove_short_words(text):
    return re.sub(r'\b\w{2,3}\b', '', text)

# Apply the function to the 'Lyrics' column
dff['Lyrics'] = dff['Lyrics'].apply(lambda x: remove_short_words(x) if isinstance(x, str) else x)
lyrics = dff['Lyrics'].dropna().tolist()
print(len(lyrics))
topic_model = BERTopic(min_topic_size=5, nr_topics=15)
topics, probabilities = topic_model.fit_transform(lyrics)
stop_words = [
    "chorus","lyric", "lyrics",'i', "im", "i'm" , "a", 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
    'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
    'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"

]
profanities = []
with open('profanity.txt', 'r') as file:
    prof_string = file.read().replace('\n', '')
    prof_tokens = prof_string.split(",")
    for token in prof_tokens:
        profanities.append(token)
stop_words.extend(profanities)
print(stop_words)

vectorizer_model = CountVectorizer(stop_words=stop_words)
topic_model.update_topics(lyrics, vectorizer_model=vectorizer_model)

topic_info = topic_model.get_topic_info()

print(topic_info)


241
['chorus', 'lyric', 'lyrics', 'i', 'im', "i'm", 'a', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's

In [107]:
topic_model.visualize_barchart(width=280, height=330, top_n_topics=8, n_words=20)


In [108]:
topic_model.visualize_topics()

In [109]:
topic_model.visualize_documents(lyrics,
# These were calculated beforehand with UMAP
topics=list(range (30)), custom_labels=True, height=600)

In [112]:
# Example usage
# Assuming reccesion_songs is your DataFrame
spotify = pd.read_csv("Billboard_Hot100_Songs_Spotify_1946-2022.csv")
start_year = 2010
end_year = 2013
result_df = filter_and_retrieve_lyrics(spotify, start_year, end_year)

# Display the resulting DataFrame
result_df

Kesha TiK ToK
Lady A Need You Now
Train Hey, Soul Sister
Katy Perry California Gurls
Usher OMG (feat. will.i.am)
B.o.B Airplanes (feat. Hayley Williams of Paramore)
Eminem Love The Way You Lie
Lady Gaga Bad Romance
Taio Cruz Dynamite
Taio Cruz Break Your Heart
B.o.B Nothin' on You (feat. Bruno Mars)
Enrique Iglesias I Like It
Young Money BedRock
Jason Derulo In My Head
Rihanna Rude Boy
Katy Perry Teenage Dream
Bruno Mars Just the Way You Are
Mike Posner Cooler Than Me - Single Mix
The Future Hit Makers Imma Be (Made Famous by Black Eyed Peas)
JAY-Z Empire State Of Mind
Usher DJ Got Us Fallin' In Love (feat. Pitbull)
Travie McCoy Billionaire (feat. Bruno Mars)
Eminem Not Afraid
Iyaz Replay
David Guetta Sexy Bitch (feat. Akon)
The Script Breakeven
Kesha Your Love Is My Drug
The Black Eyed Peas I Gotta Feeling
Owl City Fireflies
Trey Songz Say Aah (feat. Fabolous)
Drake Find Your Love
Lady Gaga Alejandro
Jason Derulo Ridin' Solo
Nelly Just A Dream
Ludacris How Low
Far East Movement Like A

,Song,Album,Album Release Date,Artist Names,Artist(s) Genres,Hot100 Ranking Year,Hot100 Rank,Song Length(ms),Spotify Link,Song Image,...,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Key,Mode,Time Signature,Lyrics
5584,TiK ToK,Animal (Expanded Edition),2010-01-01,['Kesha'],"['pop', 'dance pop']",2010,1,199693,https://open.spotify.com/track/0HPD5WQqrq7wPWR...,https://i.scdn.co/image/ab67616d00001e027a6339...,...,0.000000,0.2890,-2.718,0.1420,120.028,0.7140,2,0,4,Wake up in the morning feeling like P Diddy \n...
5585,Need You Now,Need You Now,2010-01-01,['Lady A'],"['country dawn', 'contemporary country', 'coun...",2010,2,277573,https://open.spotify.com/track/11EX5yhxr9Ihl3I...,https://i.scdn.co/image/ab67616d00001e023a2cc6...,...,0.000636,0.2000,-5.535,0.0303,107.943,0.2310,4,1,4,Picture perfect memories scattered all around ...
5586,"Hey, Soul Sister","Save Me, San Francisco (Golden Gate Edition)",2010-12-01,['Train'],"['pop', 'dance pop', 'neo mellow', 'pop rock']",2010,3,216773,https://open.spotify.com/track/4HlFJV71xXKIGcU...,https://i.scdn.co/image/ab67616d00001e026ff8bc...,...,0.000000,0.0826,-4.440,0.0431,97.012,0.7950,1,0,4,"Hey, hey, hey\r\nYour lipstick stains\r\nOn th..."
5587,California Gurls,Teenage Dream,2010-01-01,"['Katy Perry', 'Snoop Dogg']","['g funk', 'gangster rap', 'pop rap', 'rap', '...",2010,4,234653,https://open.spotify.com/track/6KOEK6SeCEZOQkL...,https://i.scdn.co/image/ab67616d00001e02d20c38...,...,0.000000,0.1800,-3.710,0.0536,125.005,0.4030,0,1,4,"Greetings loved ones, let's take a journey\n\n..."
5588,OMG (feat. will.i.am),Raymond v Raymond (Expanded Edition),2010-03-30,"['Usher', 'will.i.am']","['dance pop', 'south carolina hip hop', 'urban...",2010,5,269493,https://open.spotify.com/track/1bM50INir8voAkV...,https://i.scdn.co/image/ab67616d00001e0286b0c9...,...,0.000011,0.3600,-5.810,0.0332,129.998,0.3260,4,0,4,[feat. will.i.am]\r\n[Will.I.Am:]\r\nOh my gos...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,Brave,The Blessed Unrest,2013-07-16,['Sara Bareilles'],"['post-teen pop', 'acoustic pop', 'pop rock', ...",2013,96,220573,https://open.spotify.com/track/6Uy6K3KdmUdAfel...,https://i.scdn.co/image/ab67616d00001e02022b40...,...,0.000021,0.0425,-3.838,0.0524,185.063,0.7580,10,0,4,You can be amazing\r\nYou can turn a phrase in...
5976,Let Her Go,All The Little Lights,2012-08-28,['Passenger'],"['folk-pop', 'neo mellow']",2013,97,252733,https://open.spotify.com/track/1KxwZYyzWNyZSRy...,https://i.scdn.co/image/ab67616d00001e0242859c...,...,0.000000,0.0963,-7.346,0.0688,74.897,0.2430,7,1,4,Well you only need the light when it's burning...
5977,Runnin' Outta Moonlight,How Country Feels,2013-01-22,['Randy Houser'],"['country road', 'contemporary country', 'coun...",2013,98,193773,https://open.spotify.com/track/6enPOmd3vSFpI4j...,https://i.scdn.co/image/ab67616d00001e02e3f7c8...,...,0.000003,0.0637,-3.717,0.0393,172.024,0.7690,2,1,4,Don't you worry 'bout getting fixed up\r\nWhen...
5978,I'm Different (In the Style of 2 Chainz) [Kara...,"2013 Pop Instrumentals, Vol. 13",2012-12-10,['Karaoke All Hits'],['karaoke'],2013,99,208005,https://open.spotify.com/track/7LrjpBt1LQcLlVu...,https://i.scdn.co/image/ab67616d00001e0270383d...,...,0.103000,0.0933,-10.187,0.1270,97.493,0.0989,0,1,4,Lyrics not found


In [113]:
result_df2 = result_df[result_df['Lyrics'] != 'Lyrics not found']
result_df.to_clipboard(index=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [114]:

dff = result_df2

def remove_short_words(text):
    return re.sub(r'\b\w{2,3}\b', '', text)

# Apply the function to the 'Lyrics' column
dff['Lyrics'] = dff['Lyrics'].apply(lambda x: remove_short_words(x) if isinstance(x, str) else x)
lyrics = dff['Lyrics'].dropna().tolist()
print(len(lyrics))
topic_model = BERTopic(min_topic_size=5, nr_topics=15)
topics, probabilities = topic_model.fit_transform(lyrics)
stop_words = [
    "chorus","lyric", "lyrics",'i', "im", "i'm" , "a", 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
    'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
    'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"

]
profanities = []
with open('profanity.txt', 'r') as file:
    prof_string = file.read().replace('\n', '')
    prof_tokens = prof_string.split(",")
    for token in prof_tokens:
        profanities.append(token)
stop_words.extend(profanities)
print(stop_words)

vectorizer_model = CountVectorizer(stop_words=stop_words)
topic_model.update_topics(lyrics, vectorizer_model=vectorizer_model)

topic_info = topic_model.get_topic_info()

print(topic_info)


328
['chorus', 'lyric', 'lyrics', 'i', 'im', "i'm", 'a', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's

In [116]:
topic_model.visualize_barchart(width=280, height=330, top_n_topics=15, n_words=20)


In [117]:
topic_model.visualize_topics()

In [118]:
topic_model.visualize_documents(lyrics,
# These were calculated beforehand with UMAP
topics=list(range (30)), custom_labels=True, height=600)

In [ ]:
print(dff["Lyrics"])

4585    I  feel magic floating   \r\nBeing with  gets ...
4586    , 's a  \r\nLike seven inches from  midday \r\...
4587     Maria Maria\r\n reminds   a west side story\r...
4588    Yeah,  yeah\r\nAlright, , , \r\n's amazing   k...
4589    Somewhere there's speaking\r\n's already comin...
                              ...                        
5080    Cause     head\r\nI think about  over  over ag...
5081    Well, I 't never been  Barbie doll type\r\n, I...
5082    Down down down\r\nDown down down\r\nDown down ...
5083     Dear , 's almost June\r\nI hope this letter c...
5084    "Jack  Diane" painted a picture   life   dream...
Name: Lyrics, Length: 401, dtype: object
